In [23]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [20]:
pc = Pinecone(api_key = os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

9e37e620-0978-4938-abdb-5709dc88eeb9


In [21]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Alice Smith',
  'subject': 'Computer Science 101',
  'stars': 5,
  'review': 'Dr. Smith is an exceptional professor. Her lectures are engaging and she provides great support outside of class.'},
 {'professor': 'Dr. Bob Johnson',
  'subject': 'Mathematics for Engineers',
  'stars': 4,
  'review': 'Dr. Johnson is very knowledgeable, though his explanations can be a bit dense at times.'},
 {'professor': 'Dr. Carol Williams',
  'subject': 'Introduction to Psychology',
  'stars': 3,
  'review': "The course was interesting, but Dr. Williams's teaching style could be more interactive."},
 {'professor': 'Dr. David Brown',
  'subject': 'Advanced Physics',
  'stars': 2,
  'review': "Dr. Brown's lectures are hard to follow, and he often seems disorganized."},
 {'professor': 'Dr. Emily Davis',
  'subject': 'Ethics in Technology',
  'stars': 5,
  'review': 'Dr. Davis is fantastic. Her class discussions are thought-provoking and she is always open to questions.'},
 {'professor': 

In [27]:
processed_data = []
client = OpenAI()


for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small"
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],  # Assuming you meant "professor" instead of "prfessor"
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

In [28]:
processed_data[0]

{'values': [-0.00579157005995512,
  -0.005585058592259884,
  -0.0149921216070652,
  0.04251055791974068,
  0.022081324830651283,
  0.020010044798254967,
  0.03336240351200104,
  0.007378934882581234,
  0.0017399368807673454,
  -0.01997305639088154,
  0.025669077411293983,
  -0.007064544130116701,
  -0.009265279397368431,
  0.003436722094193101,
  0.011503001675009727,
  0.02704993076622486,
  -0.016163380816578865,
  0.011996163055300713,
  -0.0009809298207983375,
  0.04579008370637894,
  0.03427475318312645,
  -0.011120800860226154,
  0.02707458846271038,
  -0.040611881762742996,
  -0.02219228632748127,
  -0.0430530346930027,
  0.02907189354300499,
  0.04571610689163208,
  0.005393958184868097,
  0.013007145375013351,
  0.07279069721698761,
  -0.010374893434345722,
  -0.008278955705463886,
  -0.025323864072561264,
  -0.030428089201450348,
  0.04431059584021568,
  -0.011096143163740635,
  0.013931823894381523,
  0.007052215281873941,
  0.016483936458826065,
  0.0303047988563776,
  0.01

In [29]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 20}

In [30]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}